# E2. 使用 Bert + prompt 完成 SST-2 分类

&emsp; 1 &ensp; 基础介绍：`prompt-based model`简介、与`fastNLP`的结合

&emsp; 2 &ensp; 准备工作：`P-Tuning v2`原理概述、`P-Tuning v2`模型搭建

&emsp; 3 &ensp; 模型训练：加载`tokenizer`、预处理`dataset`、模型训练与分析

### 1. 基础介绍：prompt-based model 简介、与 fastNLP 的结合

&emsp; 本示例使用`GLUE`评估基准中的`SST-2`数据集，通过`prompt-based tuning`方式

&emsp; &emsp; 微调`bert-base-uncased`模型，实现文本情感的二分类，在此之前本示例

&emsp; &emsp; 将首先简单介绍提示学习模型的研究，以及与`fastNLP v1.0`结合的优势

**prompt**，**提示词**，最早出自论文[Exploiting Cloze Questions for Few Shot TC and NLI](https://arxiv.org/pdf/2001.07676.pdf)中的 **PET 模型**

&emsp; &emsp; 全称 **Pattern-Exploiting Training**，虽然文中并没有提到`prompt`的说法，但仍被视为开山之作

&emsp; 其大致思路包括，对于文本分类任务，假定输入文本为`" X . "`，设计**输入模板 template**，**后来被称为 prompt**

&emsp; &emsp; 将输入重构为`" X . It is [MASK] . "`，**诱导或刺激语言模型在 [MASK] 位置生成含有情感倾向的词汇**

&emsp; &emsp; 接着将该词汇**输入分类器中**，**后来被称为 verbalizer**，从而得到该语句对应的情感倾向，实现文本分类

&emsp; 其主要贡献在于，通过构造`prompt`，诱导/刺激预训练模型生成期望适应下游任务特征，适合少样本学习的需求

<img src="./figures/E2-fig-pet-model.png" align="center"></img>

**prompt-based tuning**，**基于提示的微调**，将`prompt`应用于**参数高效微调**，**parameter-efficient tuning**

&emsp; 通过**设计模板调整模型输入**或者**调整模型内部状态**，**固定预训练模型**，**诱导/刺激模型**调整输出以适应

&emsp; 当前任务，极大降低了训练开销，也省去了`verbalizer`的构造，更多参考[prompt综述](https://arxiv.org/pdf/2107.13586.pdf)、[DeltaTuning综述](https://arxiv.org/pdf/2203.06904.pdf)

&emsp; &emsp; 以下列举些经典的`prompt-based tuning`案例，简单地介绍下`prompt-based tuning`的脉络

&emsp; **案例一**：**PrefixTuning**，详细内容参考[PrefixTuning论文](https://arxiv.org/pdf/2101.00190.pdf)

&emsp; &emsp; 其主要贡献在于，**提出连续的、非人工构造的、任务导向的 prompt**，即**前缀 prefix**，**调整**

&emsp; &emsp; &emsp; **模型内部更新状态**，诱导模型在特定任务下生成期望目标，降低优化难度，提升微调效果

&emsp; &emsp; 其主要研究对象，是`GPT2`和`BART`，主要面向生成任务`NLG`，如`table-to-text`和摘要

&emsp; **案例二**：**P-Tuning v1**，详细内容参考[P-Tuning-v1论文](https://arxiv.org/pdf/2103.10385.pdf)

&emsp; &emsp; 其主要贡献在于，**通过连续的、非人工构造的 prompt 调整模型输入**，取代原先基于单词设计的

&emsp; &emsp; &emsp; 但离散且不易于优化的`prompt`；同时也**证明了 GPT2 在语言理解任务上仍然是可以胜任的**

&emsp; &emsp; 其主要研究对象，是`GPT2`，主要面向知识探测`knowledge probing`和自然语言理解`NLU`

&emsp; **案例三**：**PromptTuning**，详细内容参考[PromptTuning论文](https://arxiv.org/pdf/2104.08691.pdf)

&emsp; &emsp; 其主要贡献在于，通过连续的`prompt`调整模型输入，**证明了 prompt-based tuning 的效果**

&emsp; &emsp; &emsp; **随模型参数量的增加而提升**，最终**在 10B 左右追上了全参数微调 fine-tuning 的效果**

&emsp; &emsp; 其主要面向自然语言理解`NLU`，通过为每个任务定义不同的`prompt`，从而支持多任务语境

通过上述介绍可以发现`prompt-based tuning`只是模型微调方式，独立于预训练模型基础`backbone`

&emsp; 目前，加载预训练模型的主流方法是使用**transformers 模块**，而实现微调的框架则

&emsp; &emsp; 可以是`pytorch`、`paddle`、`jittor`等，而不同框架间又存在不兼容的问题

&emsp; 因此，**使用 fastNLP v1.0 实现 prompt-based tuning**，可以**很好地解决 paddle 等框架**

&emsp; &emsp; **和 transformers 模块之间的桥接**（`transformers`模块基于`pytorch`实现）

本示例仍使用了`tutorial-E1`的`SST-2`数据集、`distilbert-base-uncased`模型（便于比较

&emsp; 使用`pytorch`框架，通过将连续的`prompt`与`model`拼接，解决`SST-2`二分类任务

In [1]:
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset

import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

import sys
sys.path.append('..')

import fastNLP
from fastNLP import Trainer
from fastNLP.core.metrics import Accuracy

print(transformers.__version__)

task = 'sst2'
model_checkpoint = 'distilbert-base-uncased'  # 'bert-base-uncased'

4.18.0


### 2. 准备工作：P-Tuning v2 原理概述、P-Tuning v2 模型搭建

&emsp; 本示例使用`P-Tuning v2`作为`prompt-based tuning`与`fastNLP v1.0`结合的案例

&emsp; &emsp; 以下首先简述`P-Tuning v2`的论文原理，并由此引出`fastNLP v1.0`的代码实践

**P-Tuning v2**出自论文[Prompt Tuning Can Be Comparable to Fine-tuning Universally Across Scales and Tasks](https://arxiv.org/pdf/2110.07602.pdf)

&emsp; 其主要贡献在于，**在 PrefixTuning 等深度提示学习基础上**，**提升了其在分类标注等 NLU 任务的表现**

&emsp; &emsp; 并使之在中等规模模型，主要是**参数量在 100M-1B 区间的模型上**，**获得与全参数微调相同的效果**

&emsp; 其结构如图所示，通过**在输入序列的分类符 [CLS] 之前**，**加入前缀序列**（**序号对应嵌入是待训练的连续值向量**

&emsp; &emsp; **刺激模型在新任务下**，从`[CLS]`对应位置，**输出符合微调任务的输出**，从而达到适应微调任务的目的

<img src="./figures/E2-fig-p-tuning-v2-model.png" align="center"></img>

本示例使用`bert-base-uncased`模型，作为`P-Tuning v2`的基础`backbone`，设置`requires_grad=False`

&emsp; &emsp; 固定其参数不参与训练，**设置 pre_seq_len 长的 prefix_tokens 作为输入的提示前缀序列**

&emsp; **使用基于 nn.Embedding 的 prefix_encoder 为提示前缀嵌入**，通过`get_prompt`函数获取，再将之

&emsp; &emsp; 拼接至批量内每笔数据前得到`inputs_embeds`，同时更新自注意力掩码`attention_mask`

&emsp; 将`inputs_embeds`、`attention_mask`和`labels`输入`backbone`，**得到输出包括 loss 和 logits**

In [2]:
class SeqClsModel(nn.Module):
    def __init__(self, model_checkpoint, num_labels, pre_seq_len):
        nn.Module.__init__(self)
        self.num_labels = num_labels
        self.back_bone = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, 
                                                                            num_labels=num_labels)
        self.embeddings = self.back_bone.get_input_embeddings()

        for param in self.back_bone.parameters():
            param.requires_grad = False
        
        self.pre_seq_len = pre_seq_len
        self.prefix_tokens = torch.arange(self.pre_seq_len).long()
        self.prefix_encoder = nn.Embedding(self.pre_seq_len, self.embeddings.embedding_dim)
    
    def get_prompt(self, batch_size):
        prefix_tokens = self.prefix_tokens.unsqueeze(0).expand(batch_size, -1).to(self.back_bone.device)
        prompts = self.prefix_encoder(prefix_tokens)
        return prompts

    def forward(self, input_ids, attention_mask, labels=None):
        
        batch_size = input_ids.shape[0]
        raw_embedding = self.embeddings(input_ids)
        
        prompts = self.get_prompt(batch_size=batch_size)
        inputs_embeds = torch.cat((prompts, raw_embedding), dim=1)
        prefix_attention_mask = torch.ones(batch_size, self.pre_seq_len).to(self.back_bone.device)
        attention_mask = torch.cat((prefix_attention_mask, attention_mask), dim=1)

        outputs = self.back_bone(inputs_embeds=inputs_embeds, 
                                 attention_mask=attention_mask, labels=labels)
        return outputs

    def train_step(self, input_ids, attention_mask, labels):
        loss = self(input_ids, attention_mask, labels).loss
        return {'loss': loss}

    def evaluate_step(self, input_ids, attention_mask, labels):
        pred = self(input_ids, attention_mask, labels).logits
        pred = torch.max(pred, dim=-1)[1]
        return {'pred': pred, 'target': labels}

接着，通过确定分类数量初始化模型实例，同时调用`torch.optim.AdamW`模块初始化优化器

&emsp; 根据`P-Tuning v2`论文：`Generally, simple classification tasks prefer shorter prompts (less than 20)`

&emsp; 此处`pre_seq_len`参数设定为`20`，学习率相应做出调整，其他内容和`tutorial-E1`中的内容一致

In [3]:
model = SeqClsModel(model_checkpoint=model_checkpoint, num_labels=2, pre_seq_len=20)

optimizers = AdamW(params=model.parameters(), lr=1e-2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

### 3. 模型训练：加载 tokenizer、预处理 dataset、模型训练与分析

&emsp; 本示例沿用`tutorial-E1`中的数据集，即使用`GLUE`评估基准中的`SST-2`数据集

&emsp; &emsp; 以`bert-base-uncased`模型作为基准，基于`P-Tuning v2`方式微调

&emsp; &emsp; 数据集加载相关代码流程见下，内容和`tutorial-E1`中的内容基本一致

首先，使用`datasets.load_dataset`加载数据集，使用`transformers.AutoTokenizer`

&emsp; 构建`tokenizer`实例，通过`dataset.map`使用`tokenizer`将文本替换为词素序号序列

In [4]:
from datasets import load_dataset, load_metric

dataset = load_dataset('glue', task)

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Reusing dataset glue (/remote-home/xrliu/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
def preprocess_function(examples):
    return tokenizer(examples['sentence'], truncation=True)

encoded_dataset = dataset.map(preprocess_function, batched=True)

Loading cached processed dataset at /remote-home/xrliu/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-294e481a713c5754.arrow
Loading cached processed dataset at /remote-home/xrliu/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-ed9d9258aaf0fb54.arrow
Loading cached processed dataset at /remote-home/xrliu/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-f44c5576b89f9e6b.arrow


然后，定义`SeqClsDataset`类、定义校对函数`collate_fn`，这里沿用`tutorial-E1`中的内容

&emsp; 同样需要注意/强调的是，**\_\_getitem\_\_ 函数的返回值必须和原始数据集中的属性对应**

&emsp; **collate_fn 函数的返回值必须和 train_step 和 evaluate_step 函数的参数匹配**

In [6]:
class SeqClsDataset(Dataset):
    def __init__(self, dataset):
        Dataset.__init__(self)
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, item):
        item = self.dataset[item]
        return item['input_ids'], item['attention_mask'], [item['label']] 

def collate_fn(batch):
    input_ids, atten_mask, labels = [], [], []
    max_length = [0] * 3
    for each_item in batch:
        input_ids.append(each_item[0])
        max_length[0] = max(max_length[0], len(each_item[0]))
        atten_mask.append(each_item[1])
        max_length[1] = max(max_length[1], len(each_item[1]))
        labels.append(each_item[2])
        max_length[2] = max(max_length[2], len(each_item[2]))

    for i in range(3):
        each = (input_ids, atten_mask, labels)[i]
        for item in each:
            item.extend([0] * (max_length[i] - len(item)))
    return {'input_ids': torch.cat([torch.tensor([item]) for item in input_ids], dim=0),
            'attention_mask': torch.cat([torch.tensor([item]) for item in atten_mask], dim=0),
            'labels': torch.cat([torch.tensor(item) for item in labels], dim=0)}

再然后，分别对`tokenizer`处理过的训练集数据、验证集数据，进行预处理和批量划分

In [7]:
dataset_train = SeqClsDataset(encoded_dataset['train'])
dataloader_train = DataLoader(dataset=dataset_train, 
                              batch_size=32, shuffle=True, collate_fn=collate_fn)
dataset_valid = SeqClsDataset(encoded_dataset['validation'])
dataloader_valid = DataLoader(dataset=dataset_valid, 
                              batch_size=32, shuffle=False, collate_fn=collate_fn)

最后，使用之前完成的`dataloader_train`和`dataloader_valid`，定义训练模块`trainer`

In [8]:
trainer = Trainer(
    model=model,
    driver='torch',
    device=1,  # [0, 1],
    n_epochs=10,
    optimizers=optimizers,
    train_dataloader=dataloader_train,
    evaluate_dataloaders=dataloader_valid,
    metrics={'acc': Accuracy()}
)

&emsp; 使用`trainer.run`方法训练模型，同样每次只对验证集中的`10`个`batch`进行评估

In [9]:
trainer.run(num_eval_batch_per_dl=10)

[22:53:00] INFO     Running evaluator sanity check for 2 batches.              ]8;id=406635;file://../fastNLP/core/controllers/trainer.py\trainer.py]8;;\:]8;id=951504;file://../fastNLP/core/controllers/trainer.py#592\592]8;;\

Output()

Output()

----------------------------- Eval. results on Epoch:1, Batch:0 -----------------------------

{
  "acc#acc": 0.540625,
  "total#acc": 320.0,
  "correct#acc": 173.0
}

...

{
  "acc#acc": 0.665625,
  "total#acc": 320.0,
  "correct#acc": 213.0
}

----------------------------- Eval. results on Epoch:9, Batch:0 -----------------------------

{
  "acc#acc": 0.659375,
  "total#acc": 320.0,
  "correct#acc": 211.0
}

---------------------------- Eval. results on Epoch:10, Batch:0 -----------------------------

{
  "acc#acc": 0.696875,
  "total#acc": 320.0,
  "correct#acc": 223.0
}

可以发现，其效果远远逊色于`fine-tuning`，这是因为`P-Tuning v2`虽然能够适应参数量

&emsp; 在`100M-1B`区间的模型，但是，**distilbert-base 的参数量仅为 66M**，无法触及其下限

另一方面，**fastNLP v1.0 不支持 jupyter 多卡**，所以无法在笔者的电脑/服务器上，完成

&emsp; 合适规模模型的学习，例如`110M`的`bert-base`模型，以及`340M`的`bert-large`模型

In [10]:
trainer.evaluator.run()

Output()

{'acc#acc': 0.737385, 'total#acc': 872.0, 'correct#acc': 643.0}